# 使用数学优化进行共享单车重调配

共享单车系统已经成为全球城市居民和游客日常通勤的有效方式。

作为最大的共享单车网络,纽约市的 Citi-Bike 在2022年7月拥有1,588个活跃站点和25,575辆活跃单车。

2022年7月完成了超过300万次骑行,覆盖纽约市/霍博肯/泽西市,拥有约15万名活跃年度会员。

在高峰时段,许多车站对单车有很高的需求,这意味着这些站点的单车流出量大于流入量。

同时,有些站点对停车桩有很高的需求(骑手将单车归还到这些站点),这意味着它们的单车流入量大于流出量。

在高需求站点缺乏可用单车或停车桩会导致共享单车网络严重失衡,并导致客户不满和收入损失。

为了解决这个问题,需要在站点之间调配单车以在供需之间取得平衡。

## 问题陈述和解决方案

使用2022年7月纽约市和泽西地区的历史 Citi-bike 数据,我们想知道:
- 8月第一周每个站点每小时的单车需求是多少?
- 知道需求后,如何最小化销售损失?

当客户需要单车但没有可用单车时就会造成销售损失。因此,应该将单车从流入量较高的站点转移到流出量较高的站点。

所以,首先要确定每个站点在每个小时需要增加或减少的单车数量。然后,安排站点之间的实际单车转移。

在这个笔记本中,我们将专注于第一部分,并在最后讨论如何解决第二部分。
我们将使用机器学习(ML)和数学优化(MO)的结合来解决这个问题。

**解决方案**
解决方案包括两个步骤:
- **步骤 1**: 我们使用2022年7月纽约市和泽西地区的历史 Citi-bike 数据,使用 ML 模型预测8月第一周每个站点每小时的单车流入和流出量。这在 [predict_bike_flow](predict_bike_flow.ipynb) 笔记本中完成。
- **步骤 2**: 我们使用 MO 模型来决定在每个小时应该在每个站点增加或减少多少单车,以最小化总销售损失。

为确保每个人都可以使用 Gurobi 限制许可证运行笔记本,我们减少了数据规模。为此,我们将重点关注早高峰时段(上午7点至9点)的前50个站点。

前50个站点是使用 PageRank 算法选择的。

# 安装所需的包

In [ ]:
%pip install gurobipy
%pip install pandas

# 导入包

In [1]:
import datetime
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# 优化问题

## 问题定义

我们想要最小化总销售损失。每个站点每小时的销售损失可以定义为单车总需求(从该站点开始骑行的单车数量)与总供应量之间的差异。

总供应量包括在该站点结束骑行的单车数量,加上站点现有的所有单车(即库存),再加上通过调配在该小时内增加或减少的单车数量。

**假设:**
- 第一小时开始时(在我们的例子中是7点)的库存为零。
- 在任何给定时间,我们都能获得有限数量的单车,可以分配给各个站点以帮助减少不平衡(因为这个分析是在早高峰时段进行的,暂时不考虑站点之间的单车转移)。

## 加载所需数据

In [2]:
stations = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/bike_share/top_stations.csv', index_col='station')
# run locally
# stations = pd.read_csv('top_stations.csv', index_col='station')
stations.head()

,capacity,lat,lon,region
station,,,,
Cleveland Pl & Spring St,33,40.722104,-73.997249,71
Grand Army Plaza & Central Park S,96,40.764397,-73.973715,71
Broadway & E 14 St,114,40.734546,-73.990741,71
Lafayette St & E 8 St,47,40.730207,-73.991026,71
Norfolk St & Broome St,59,40.717227,-73.988021,71


In [3]:
stations_flow = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/bike_share/stations_flow.csv')
# run locally
# stations_flow = pd.read_csv('stations_flow.csv')
stations_flow['datetime'] = pd.to_datetime(stations_flow['datetime'])
stations_flow.head()

,station,datetime,start_forecast,end_forecast
0,1 Ave & E 62 St,2022-08-01 00:00:00,4.0,5.0
1,1 Ave & E 62 St,2022-08-01 01:00:00,2.0,3.0
2,1 Ave & E 62 St,2022-08-01 02:00:00,4.0,2.0
3,1 Ave & E 62 St,2022-08-01 03:00:00,2.0,2.0
4,1 Ave & E 62 St,2022-08-01 04:00:00,2.0,2.0


`stations_flow` 数据包含2022年8月前5天所有时段的预测。我们的分析针对早高峰时段,即上午7点到9点。此外,我们可以每天运行我们的MO模型。目前,我们只关注第一天,但最后我们将展示完整模型以及如何每天运行它。

In [ ]:
# 当创建新列时,Pandas会给出几个SettingWithCopyWarning警告
# 这些是误报,所以我们忽略它们
pd.options.mode.chained_assignment = None
morning_flow = stations_flow[stations_flow['datetime'].dt.hour.between(7, 9)]
morning_flow['date'] = morning_flow['datetime'].dt.date
morning_flow['time'] = morning_flow['datetime'].dt.hour
# 现在,让我们为第一天运行MO模型: 2022/08/01
flow_df = morning_flow.loc[morning_flow['date'] == datetime.date(2022, 8, 1)]
flow_df.set_index(['station', 'time'], inplace=True)
flow_df.head()

datetime  start_forecast  end_forecast  \
station         time                                                     
1 Ave & E 62 St 7    2022-08-01 07:00:00            11.0          10.0   
                8    2022-08-01 08:00:00            14.0          14.0   
                9    2022-08-01 09:00:00            17.0          20.0   
1 Ave & E 68 St 7    2022-08-01 07:00:00            17.0          32.0   
                8    2022-08-01 08:00:00            22.0          41.0   

                            date  
station         time              
1 Ave & E 62 St 7     2022-08-01  
                8     2022-08-01  
                9     2022-08-01  
1 Ave & E 68 St 7     2022-08-01  
                8     2022-08-01

## 问题公式化

让我们定义MO模型的符号。我们想为每个小时和每个站点运行模型。所以,让我们定义以下两个集合:

**集合**
- $I\quad$: 站点集合
- $T\quad$: 小时集合

我们还从 `stations` 和 `flow_df` 数据框中获取以下信息:

**参数**
- $e_{i,t}\quad$: 在t小时在站点i结束骑行的单车数量(即供应)
- $s_{i,t}\quad$: 在t小时从站点i开始骑行的单车数量(即需求)
- $c_{i}\quad$: 站点i的容量

我们知道在早高峰和交通繁忙时段,在站点之间转移单车并不容易。为了减少由于高需求站点缺乏可用单车而造成的损失,我们假设在每个小时开始时都有一小部分预备单车可以分配给各站点。我们用$N$表示:
- $N\quad$: 在给定时间可以分配给站点的预备单车数量

In [ ]:
num_bikes = 25  # N: 在给定时间可以分配给站点的预备单车数量
station_names = list(stations.index)  # 站点集合I
time_rng = morning_flow['time'].drop_duplicates().values  # 时间集合T

In [ ]:
station_time = flow_df.index  # (i,t)索引对
start_forecast = flow_df.start_forecast  # s
end_forecast = flow_df.end_forecast  # e
capacity = stations.capacity  # c

## 决策变量

首先,我们要找出在每个小时内应该在每个站点增加或减少多少单车。我们用$y_{i,t}$表示增加的单车数量,用$z_{i,t}$表示减少的单车数量:
- $y_{i,t}\quad$: 在t小时增加到站点i的单车数量
- $z_{i,t}\quad$: 在t小时从站点i移除的单车数量

另一个变量是每个站点在每个小时开始时的单车库存。由于库存取决于$y_{i,t}$和$z_{i,t}$的值,所以它也是一个决策变量。我们用$q_{i,t}$表示:
- $q_{i,t}\quad$: 在t小时开始时站点i的单车库存

为简单起见,我们假设在第一个小时开始时没有库存。在我们的例子中,这意味着上午7点时没有初始库存。

模型的目标是减少每个站点每小时的销售损失。这个值也取决于决策变量$y_{i,t}$和$z_{i,t}$的值,因此它本身就是一个决策变量。我们用$l_{i,t}$表示:
- $l_{i,t}\quad$: 在t小时站点i的销售损失

有了这些初始决策变量,我们可以开始构建MO模型:

In [ ]:
mdl = gp.Model('bike_rebalancing')
# 变量

y = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='y')
# y = mdl.addVars(station_names, time_rng, lb=0, vtype=GRB.CONTINUOUS, name='y')  # 另一种方式
z = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='z')
q = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='q')
l = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='l')

## 约束条件

首先,我们设定每小时可以在站点增加或减少的单车数量的上下限。

每个小时,我们有$s_{i,t}$辆单车从站点开始骑行,$e_{i,t}$辆单车在站点结束骑行。如果$s_{i,t} \ge e_{i,t}$,说明需求超过供应。在这种情况下,我们可以选择向这个站点增加一些单车以减少损失(我们也可以选择不增加)。需要记住的一点是,我们不能添加超过站点容量的单车。

另一方面,如果$s_{i,t} \le e_{i,t}$,我们可以选择从该站点移除一些多余的单车(我们也可以选择不移除)。但是,如果$e_{i,t} \ge s_{i,t} + c_i$,那么到达站点的单车数量将超过站点容量。在这种情况下,我们必须移除一些单车以避免溢出。

以上描述帮助我们定义$y_{i,t}$和$z_{i,t}$的边界。
对于增加单车,边界是:

\begin{align}
&0 \le y_{i,t} \le c_i &\quad \forall i \in I, t \in T \tag{1}\\
\end{align}

对于移除单车,上限是:

\begin{align}
&z_{i,t} \le \max(0, e_{i,t} - s_{i,t}) &\quad \forall i \in I, t \in T \tag{2}\\
\end{align}

下限是:

\begin{align}
&z_{i,t} \ge  e_{i,t} - s_{i,t} - c_i &\quad \forall i \in I, t \in T \tag{3}\\
\end{align}

In [ ]:
# y的上界
mdl.addConstrs((y[i, t] <= capacity[i] for i, t in station_time), 'ub_y')

# z的下界
mdl.addConstrs(
    (z[i, t] >= (end_forecast[i, t] - start_forecast[i, t] - capacity[i])
     for i, t in station_time), 'lb_z')
#     for i in station_names for t in time_rng), 'lb_z')

# z的上界
mdl.addConstrs(
        (z[i, t] <= max(0, end_forecast[i, t] - start_forecast[i, t])
         for i, t in station_time), 'ub_z');  # 在这里添加";"以阻止单元格打印约束

接下来,我们要定义如何计算销售损失。销售损失是站点的需求与所有单车供应之间的差异。
- 单车需求是指以任何形式从站点离开的所有单车。那么,需求包括哪些?
- 单车供应是指以任何形式到达站点的所有单车。那么,供应包括哪些?

当然,如果供应大于需求,就没有损失。因此,我们需要确保损失只考虑非负值。这可以通过以下方式实现:

\begin{align}
&l_{i,t} = \max(0, s_{i,t} + z_{i,t} - e_{i,t} - y_{i,t} - q_{i,t}) &\quad \forall i \in I, t \in T \tag{9}\\
\end{align}

In [ ]:
# 损失定义
mdl.addConstrs(
    (l[i, t] >= start_forecast[i, t] + z[i, t]
     - end_forecast[i, t] - y[i, t] - q[i, t]
     for i, t in station_time), 'loss_def');

我们假设在每个小时开始时有一小部分预备单车可以分配给站点。这个限制是针对增加到站点的总单车数量。

\begin{align}
&\sum_{i} y_{i,t} \le N &\quad \forall t \in T \tag{10}\\
\end{align}

In [ ]:
# 增加单车数量的限制
mdl.addConstrs((y.sum('*', t) <= num_bikes for t in time_rng), name='total_bikes');

## 目标

目标是最小化总销售损失。

$$\min \sum_{i,t} l_{i,t}$$

In [15]:
mdl.setObjective(l.sum(), GRB.MINIMIZE)

现在我们可以告诉Gurobi模型已经完成,它可以开始求解问题。

In [16]:
mdl.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 813 rows, 675 columns and 1620 nonzeros
Model fingerprint: 0xecda0867
Model has 90 general constraints
Variable types: 675 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 750 rows and 580 columns
Presolve time: 0.00s
Presolved: 63 rows, 95 columns, 201 nonzeros
Variable types: 81 continuous, 14 integer (14 binary)
Found heuristic solution: objective 40.0000000

Root relaxation: objective 1.368348e+01, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.68348    0    9   40.00000   13.68348  65.8%     -    0s
     0 

## 后处理

In [17]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in y.items():
        df.loc[k, 'bikes_added'] = v.x
        df.loc[k, 'bikes_removed'] = z[k].x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time')['bikes_added'].sum()
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

Total Loss : 40.0
Total number of bikes added in each hour:
 time
7    25
8    25
9    25
Name: bikes_added, dtype: int64


,station,time,datetime,start_forecast,end_forecast,date,capacity,bikes_added,bikes_removed,loss_sale,beginning_inventory
0,1 Ave & E 62 St,7,2022-08-01 07:00:00,11.0,10.0,2022-08-01,45,0,0,1,0
1,1 Ave & E 62 St,8,2022-08-01 08:00:00,14.0,14.0,2022-08-01,45,0,0,0,0
2,1 Ave & E 62 St,9,2022-08-01 09:00:00,17.0,20.0,2022-08-01,45,0,0,0,0
3,1 Ave & E 68 St,7,2022-08-01 07:00:00,17.0,32.0,2022-08-01,62,0,15,0,0
4,1 Ave & E 68 St,8,2022-08-01 08:00:00,22.0,41.0,2022-08-01,62,0,19,0,0
5,1 Ave & E 68 St,9,2022-08-01 09:00:00,28.0,48.0,2022-08-01,62,0,0,0,0
6,1 Ave & E 78 St,7,2022-08-01 07:00:00,9.0,7.0,2022-08-01,57,7,0,0,0
7,1 Ave & E 78 St,8,2022-08-01 08:00:00,18.0,5.0,2022-08-01,57,8,0,0,5
8,1 Ave & E 78 St,9,2022-08-01 09:00:00,18.0,7.0,2022-08-01,57,11,0,0,0
9,10 Ave & W 14 St,7,2022-08-01 07:00:00,3.0,5.0,2022-08-01,55,0,2,0,0


## 整合所有内容

**集合**
- $I\quad$: 站点集合
- $T\quad$: 小时集合 

**参数**
- $e_{i,t}\quad$: 在t小时在站点i结束骑行的单车数量(即供应)
- $s_{i,t}\quad$: 在t小时从站点i开始骑行的单车数量(即需求)
- $c_{i}\quad$: 站点i的容量
- $N\quad$: 在给定时间可以分配给站点的预备单车数量

**变量**
- $y_{i,t}\quad$: 在t小时增加到站点i的单车数量
- $z_{i,t}\quad$: 在t小时从站点i移除的单车数量
- $q_{i,t}\quad$: 在t小时开始时站点i的单车库存
- $l_{i,t}\quad$: 在t小时站点i的销售损失
- $a_{i,t}\quad$: 定义库存与其他变量关系的辅助变量。我们在代码中使用它

\begin{align}
&\min \sum_{i,t} l_{i,t}&\\
\mbox{s.t: }\\
&y_{i,t} \le c_i &\quad \forall i \in I, t \in T \tag{1}\\
&z_{i,t} \ge \max(0, e_{i,t} - s_{i,t}) &\quad \forall i \in I, t \in T \tag{2}\\
&z_{i,t} \le  e_{i,t} - s_{i,t} - c_i &\quad \forall i \in I, t \in T \tag{3}\\
&q_{i,t_0} = 0 &\quad \forall i \in I \tag{4}\\
&a_{i,t} = e_{i,t-1} + y_{i,t-1} + q_{i,t-1} - s_{i,t-1} - z_{i,t-1} &\quad \forall i \in I, t \in T/t_0 \tag{5}\\
&q_{i,t} = \max(0, a_{i,t}) &\quad \forall i \in I, t \in T/t_0 \tag{6}\\
&q_{i,t} \le c_i &\quad \forall i \in I, t \in T \tag{7}\\
&l_{i,t} = \max(0, s_{i,t} + z_{i,t} - e_{i,t} - y_{i,t} - q_{i,t}) &\quad \forall i \in I, t \in T \tag{8}\\
&\sum_{i} y_{i,t} \le N &\quad \forall t \in T \tag{9}\\
&y_{i,t}, z_{i,t}, q_{i,t}, l_{i,t} \ge0 &\quad \forall i \in I, t \in T \tag{10}\\
&a_{i,t} \quad \mbox{urs} &\quad \forall i \in I, t \in T \tag{11}\\
\end{align}

In [ ]:
def bike_rebalancing(flow_df, num_bikes):
    station_time = flow_df.index  # (i,t)索引对
    start_forecast = flow_df.start_forecast  # s
    end_forecast = flow_df.end_forecast  # e
    capacity = stations.capacity  # c
    mdl = gp.Model('bike_rebalancing')

    # 变量
    y = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='y')
    z = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='z')
    q = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='q')
    l = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='l')
    a = mdl.addVars(station_time, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='a')

    # 约束条件
    mdl.addConstrs((y[i, t] <= capacity[i] for i, t in station_time), 'ub_y')

    mdl.addConstrs(
        (z[i, t] >= (end_forecast[i, t] - start_forecast[i, t] - capacity[i])
         for i, t in station_time), 'lb_z')
    
    mdl.addConstrs(
        (z[i, t] <= max(0, end_forecast[i, t] - start_forecast[i, t])
         for i, t in station_time), 'ub_z')

    t0 = 7
    mdl.addConstrs((q[i, t0] == 0 for i in stations.index), name='initial_inv')

    mdl.addConstrs((a[i, t] == end_forecast[i, t - 1] + y[i, t - 1] + q[i, t - 1]
                    - start_forecast[i, t - 1] - z[i, t - 1]
                    for i, t in station_time if t != t0), name='aux_def')

    mdl.addConstrs((q[i, t] == gp.max_(a[i, t], 0) for i, t in station_time if t != t0), name='inv_def')

    mdl.addConstrs((q[i, t] <= capacity[i] for i, t in station_time), 'ub_inv')

    mdl.addConstrs((l[i, t] >= start_forecast[i, t] + z[i, t]
                    - end_forecast[i, t] - y[i, t] - q[i, t] for i, t in station_time), 'loss_def')

    mdl.addConstrs((y.sum('*', t) <= num_bikes for t in time_rng), name='total_bikes')

    # 目标
    mdl.setObjective(l.sum(), GRB.MINIMIZE)
    mdl.optimize()

    # 创建输出
    df = pd.DataFrame()
    if mdl.status == GRB.Status.OPTIMAL:
        df = flow_df.copy()
        df = df.merge(stations[['capacity']], left_on='station', right_index=True)
        df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
        for k, v in y.items():
            df.loc[k, 'bikes_added'] = v.x
            df.loc[k, 'bikes_removed'] = z[k].x
            df.loc[k, 'beginning_inventory'] = q[k].x
            df.loc[k, 'loss_sale'] = l[k].x
        df.reset_index(inplace=True)
        print(f'Total Loss : {mdl.objVal}')
        total_bikes_added = df.groupby('time')['bikes_added'].sum()
        print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
    else:
        print('Could not find a feasible solution!')
    return df

In [ ]:
# 每天运行模型
all_outputs = []
g = morning_flow.set_index(['station', 'time']).groupby('date')
for x in g.groups:
    df = g.get_group(x)
    odf = bike_rebalancing(df, num_bikes)
    all_outputs.append(odf)
output_df = pd.concat(all_outputs)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 813 rows, 675 columns and 1620 nonzeros
Model fingerprint: 0xecda0867
Model has 90 general constraints
Variable types: 675 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 750 rows and 580 columns
Presolve time: 0.00s
Presolved: 63 rows, 95 columns, 201 nonzeros
Variable types: 81 continuous, 14 integer (14 binary)
Found heuristic solution: objective 40.0000000

Root relaxation: objective 1.368348e+01, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.68348    0    9   40.00000   13.68348  65.8%     -    0s
     0 

In [20]:
output_df.head(10)

,station,time,datetime,start_forecast,end_forecast,date,capacity,bikes_added,bikes_removed,loss_sale,beginning_inventory
0,1 Ave & E 62 St,7,2022-08-01 07:00:00,11.0,10.0,2022-08-01,45,0.0,0,1.0,0.0
1,1 Ave & E 62 St,8,2022-08-01 08:00:00,14.0,14.0,2022-08-01,45,0.0,0,0.0,0.0
2,1 Ave & E 62 St,9,2022-08-01 09:00:00,17.0,20.0,2022-08-01,45,0.0,0,0.0,0.0
3,1 Ave & E 68 St,7,2022-08-01 07:00:00,17.0,32.0,2022-08-01,62,0.0,15,0.0,0.0
4,1 Ave & E 68 St,8,2022-08-01 08:00:00,22.0,41.0,2022-08-01,62,0.0,19,0.0,0.0
5,1 Ave & E 68 St,9,2022-08-01 09:00:00,28.0,48.0,2022-08-01,62,0.0,0,0.0,0.0
6,1 Ave & E 78 St,7,2022-08-01 07:00:00,9.0,7.0,2022-08-01,57,7.0,0,0.0,0.0
7,1 Ave & E 78 St,8,2022-08-01 08:00:00,18.0,5.0,2022-08-01,57,8.0,0,0.0,5.0
8,1 Ave & E 78 St,9,2022-08-01 09:00:00,18.0,7.0,2022-08-01,57,11.0,0,0.0,0.0
9,10 Ave & W 14 St,7,2022-08-01 07:00:00,3.0,5.0,2022-08-01,55,0.0,2,0.0,0.0


# 模型改进

查看 `output_df`,你可能会注意到有些站点在每个小时都有单车转移(增加或减少)。更糟糕的是,有些站点在一个小时内移除一些单车,但在下一个小时又增加单车。我们知道这些增加/减少会消耗时间和金钱。我们该如何避免这种情况?

一种方法是为使用卡车(或自行车拖车)转移单车引入固定成本,然后将这一项添加到目标函数中。通过与转移相关的成本,模型会被激励使用更少的转移次数。

所以首先,我们应该计算发生转移的次数。每当向站点增加或从站点移除单车时,就发生了一次转移。因此,我们需要一种方法将单车的增加和移除与转移联系起来。一种方法是引入两个新的二进制变量:

- $x_{i,t}\quad$: 如果在t小时向站点i增加任何单车则为1;否则为0
- $w_{i,t}\quad$: 如果在t小时从站点i移除任何单车则为1;否则为0

接下来,我们需要建立$y_{i,t}$与$x_{i,t}$以及$z_{i,t}$与$w_{i,t}$之间的关系。基本上,我们想要表达:
如果$y_{i,t} \ge 0$,则$x_{i,t} = 1$,如果$z_{i,t} \ge 0$,则$w_{i,t} = 1$。

我们引入以下两个约束:

\begin{align}
&y_{i,t} \le M x_{i,t} &\quad \forall i \in I, t \in T \tag{11}\\
&z_{i,t} \le M w_{i,t} &\quad \forall i \in I, t \in T \tag{12}\\
\end{align}

其中$M$是一个很大的数。

约束11确保如果$y_{i,t}\ge 0$,则$x_{i,t} = 1$。但仅凭这个约束无法使$y_{i,t} \le 0$时$x_{i,t} =0$。约束12也是如此。它确保$z_{i,t} \ge 0$使$w_{i,t} = 1$。但是,它不能强制$z_{i,t} \ge 0$时$w_{i,t} = 0$。

这可以通过目标函数来实现。

转移的总次数等于$x_{i,t}$和$w_{i,t}$的总和,我们的目标是最小化转移次数。所以,我们将这些项添加到目标函数中。换句话说,我们的新目标函数是:

$$\min \sum_{i,t} (l_{i,t} + x_{i,t} + w_{i,t})$$

由于需要最小化总转移次数,目标函数试图使$x_{i,t}$和$w_{i,t}$尽可能小(在这种情况下为零)。结合约束11和12,这意味着在$x_{i,t}$和$w_{i,t}$可以取0或1的情况下,目标函数强制它们取值为0。此外,由于任何额外的转移都会导致$x_{i,t}$或$w_{i,t}$为1,该模型会被激励减少转移次数以最小化目标函数。

当然,你可以通过为目标函数中的每一项设置系数(将它们视为成本)来使这个模型更加通用。

In [ ]:
x = mdl.addVars(station_time, vtype=GRB.BINARY, name='x')  # 如果 y_{i,t} >= 0 则为1
w = mdl.addVars(station_time, vtype=GRB.BINARY, name='w')  # 如果 z_{i,t} >= 0 则为1

big_m = 1000  # 一个很大的数
# y和x之间的关系
mdl.addConstrs((y[i, t] <= big_m * x[i, t] for i, t in station_time), 'rel_y_x')
# z和w之间的关系
mdl.addConstrs((z[i, t] <= big_m * w[i, t] for i, t in station_time), 'rel_z_w')

# 新的目标函数
obj = l.sum() + (x.sum() + w.sum())
mdl.setObjective(obj, GRB.MINIMIZE)
mdl.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1083 rows, 945 columns and 2160 nonzeros
Model fingerprint: 0x361303a6
Model has 90 general constraints
Variable types: 675 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve produced solution with objective 52 (0.01s)
MIP start from previous solve produced solution with objective 49 (0.03s)
Loaded MIP start from previous solve with objective 49

Presolve removed 974 rows and 789 columns
Presolve time: 0.00s
Presolved: 109 rows, 156 columns, 320 nonzeros
Variable types: 99 continuous, 57 integer (57 binary)

Root relaxation: objective 1.700045e+01, 114 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    

## 后处理

In [22]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in y.items():
        df.loc[k, 'bikes_added'] = v.x
        df.loc[k, 'bikes_removed'] = z[k].x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time')['bikes_added'].sum()
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

Total Loss : 49.0
Total number of bikes added in each hour:
 time
7    25
8    25
9    25
Name: bikes_added, dtype: int64


,station,time,datetime,start_forecast,end_forecast,date,capacity,bikes_added,bikes_removed,loss_sale,beginning_inventory
0,1 Ave & E 62 St,7,2022-08-01 07:00:00,11.0,10.0,2022-08-01,45,0,0,1.0,0
1,1 Ave & E 62 St,8,2022-08-01 08:00:00,14.0,14.0,2022-08-01,45,0,0,0.0,0
2,1 Ave & E 62 St,9,2022-08-01 09:00:00,17.0,20.0,2022-08-01,45,0,0,0.0,0
3,1 Ave & E 68 St,7,2022-08-01 07:00:00,17.0,32.0,2022-08-01,62,0,0,0.0,0
4,1 Ave & E 68 St,8,2022-08-01 08:00:00,22.0,41.0,2022-08-01,62,0,0,0.0,15
5,1 Ave & E 68 St,9,2022-08-01 09:00:00,28.0,48.0,2022-08-01,62,0,0,0.0,34
6,1 Ave & E 78 St,7,2022-08-01 07:00:00,9.0,7.0,2022-08-01,57,0,0,2.0,0
7,1 Ave & E 78 St,8,2022-08-01 08:00:00,18.0,5.0,2022-08-01,57,18,0,0.0,0
8,1 Ave & E 78 St,9,2022-08-01 09:00:00,18.0,7.0,2022-08-01,57,6,0,0.0,5
9,10 Ave & W 14 St,7,2022-08-01 07:00:00,3.0,5.0,2022-08-01,55,0,0,0.0,0


# 附加内容

## 场景分析

许多MO问题中一个重要的要求是场景分析或假设分析。
一般来说,在假设分析中,我们想知道在不同场景下解决方案如何变化。
想想我们这个案例。
- 如果预备单车数量增加或减少10%,解决方案会如何变化?
- 在改进的模型中,如果销售损失的成本发生变化会怎样?增加或移除单车的访问成本呢?
- 如果在最繁忙的站点附近新增一个站点会怎样?
- 如果我们想确保每个站点在每个小时开始时至少有2辆可用单车呢?

我们这里的模型还是一个简单的模型。但你可以想象场景分析能提供的价值。它可以通过创建和比较不同场景并评估其结果来帮助你回答许多问题,从而评估它们对业务目标的影响。要了解更多信息,你可以查看Gurobi的[多场景示例](https://www.gurobi.com/documentation/9.5/examples/multiscenario_py.html)。

## 这个问题在现实中是如何解决的?

在知道每个站点需要多少单车后,需要将单车从一个站点实际移动到另一个站点。

在高峰时段,交通已经很拥堵。所以使用自行车拖车(通常可以容纳5辆自行车)来移动单车。

在非高峰时段(主要是夜间),使用卡车转移单车。

无论哪种情况,都应该从单车流入量较大的站点移除一些单车,并转移到单车流出量较大的站点,以达到平衡。
这个问题,即卡车需要从一个站点到另一个站点提取或投放单车,本身就是另一个数学优化问题。

在这个问题中,我们需要确保所有需要提取或投放的站点都在特定时间窗口内被访问,目标可以是最小化卡车数量或最小化运输成本(例如,燃料成本加上使用卡车的成本)。这个问题是著名的带提取和投放的车辆路径问题(VRP)的一个变体。
要了解更多信息,请查看[此网络研讨会](https://www.gurobi.com/resource/how-to-synchronize-complex-routing-operations-synched-vrps-with-gurobi/)